In [ ]:
# Definindo um limiar para considerar uma epidemia
limiar_epidemia = 50  # Exemplificando com 50 casos por 100.000 habitantes

# Adicionando o cálculo da taxa de incidência e a verificação de epidemia
for pais in paises:
    # Filtrar os dados para o país
    df_pais = df[df['Local'] == pais]
    
    # Selecionar as features e o target como no código original
    features = df_pais[['BCG', 'Hib3', 'DTP3', 'Saneamento Seguro (%)', 
                        'Numero Total de Imigrantes', 'Gini (Indice de Desigualdade no Pais)', 
                        'Populacao', 'Taxa de Extrema Pobreza (%)', 'Numero de Usuario de Internet']].values
    target = df_pais['Numero de Casos de Infeccoes Respiratorios e Tuberculose'].values.reshape(-1, 1)

    # Normalizando os dados e criando os conjuntos de treino e teste como no código original
    scaler_features = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler_features.fit_transform(features)
    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaled_target = scaler_target.fit_transform(target)

    X, y = create_dataset(scaled_features, scaled_target, n_steps)

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1)) 
    model.compile(optimizer='adam', loss='mse')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=400, verbose=0, validation_split=0.1, callbacks=[early_stopping])

    # Previsões para o conjunto de teste
    y_pred = model.predict(X_test)
    y_pred = scaler_target.inverse_transform(y_pred)
    y_test = scaler_target.inverse_transform(y_test.reshape(-1, 1))

    mse = mean_squared_error(y_test, y_pred)
    mse_paises[pais] = mse

    # Previsão futura
    last_sequence = scaled_features[-n_steps:]
    predictions = []
    n_features = 9
    for _ in range(5):  # Prever os próximos 5 anos
        last_sequence = last_sequence.reshape((1, n_steps, X_train.shape[2]))
        next_value = model.predict(last_sequence)
        predictions.append(next_value[0, 0])
        next_value_expanded = np.repeat(np.expand_dims(next_value, axis=1), n_features, axis=2)
        last_sequence = np.concatenate((last_sequence[:, 1:, :], next_value_expanded), axis=1)

    predictions = scaler_target.inverse_transform(np.array(predictions).reshape(-1, 1))
    previsoes_paises[pais] = predictions.flatten()

    # Cálculo da taxa de incidência
    populacoes = df_pais['Populacao'].values[-len(predictions):]  # Usar população correspondente ao ano da previsão
    taxas_incidencia = (predictions.flatten() / populacoes) * 100000

    # Verificando se o país estará em epidemia
    em_epidemia = taxas_incidencia > limiar_epidemia
    for i, em_epidemia_ano in enumerate(em_epidemia):
        status_epidemia = "Epidemia" if em_epidemia_ano else "Não em Epidemia"
        print(f'Ano: {inicio_previsao + i} - Previsão de Incidência: {taxas_incidencia[i]:.2f} - Status: {status_epidemia}')

    # Plotando as previsões e a taxa de incidência
    plt.figure(figsize=(14, 8))
    plt.plot(df_pais['Ano'], df_pais['Numero de Casos de Infeccoes Respiratorios e Tuberculose'], label='Dados Reais')
    plt.plot(future_years, previsoes_paises[pais], label='Previsões Futuras', color='red')
    plt.plot(future_years, taxas_incidencia, label='Taxa de Incidência (por 100k hab)', color='green')
    plt.axhline(y=limiar_epidemia, color='orange', linestyle='--', label='Limiar de Epidemia')
    plt.xlabel('Ano')
    plt.ylabel('Número de Casos e Taxa de Incidência')
    plt.title(f'Previsão de Casos e Taxa de Incidência para {pais} ({inicio_previsao} - {fim_previsao})')
    plt.legend()
    plt.show()


In [2]:
import pandas as pd
import numpy as np

# Dados fictícios de exemplo
dados_historicos = {
    'ano': [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 
            2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
    'casos': [62784899, 63249957, 63660428, 64020301, 
            64334928, 64610502, 64820118, 64941261, 
            64992516, 64994141, 64965154, 64889004, 
            64744480, 64542620, 64291488, 64004934, 
            63626040, 63150966, 62673404, 62288579]  
            # Número de casos esperados por ano
}

dados_atual = {
    'ano': [2020],
    'casos_observados': [64359713]  # Número de casos observados no ano atual
}

# Criação dos DataFrames
df_historicos = pd.DataFrame(dados_historicos)
df_atual = pd.DataFrame(dados_atual)

# Cálculo da média dos casos históricos
media_casos_historicos = df_historicos['casos'].mean()

# Cálculo da taxa esperada para o ano atual (por exemplo, usando a média histórica)
taxa_esperada = media_casos_historicos

# Cálculo da razão observada/esperada
df_atual['taxa_esperada'] = taxa_esperada
df_atual['razao_observada_esperada'] = df_atual['casos_observados'] / df_atual['taxa_esperada']

# Exibindo resultados
print("Dados Atuais:")
print(df_atual)

# Verificação se é uma epidemia
limite_critical = 2  # Limite crítico para considerar uma epidemia
if df_atual['razao_observada_esperada'].iloc[0] >= limite_critical:
    print("A população pode estar em epidemia.")
else:
    print("A população não está em epidemia.")


Dados Atuais:
    ano  casos_observados  taxa_esperada  razao_observada_esperada
0  2020          64359713     64079286.0                  1.004376
A população não está em epidemia.


In [1]:
import numpy as np

# Dados históricos de taxas de incidência (casos por 100.000 habitantes)
dados_incidencia = np.array([30, 25, 28, 32, 40, 45, 29, 35, 33, 42])

# Cálculo da média
media = np.mean(dados_incidencia)

# Cálculo do desvio padrão
desvio_padrao = np.std(dados_incidencia)

# Definindo o fator ajustável (k)
k = 1  # Você pode alterar para 2 para maior sensibilidade

# Cálculo do limiar epidêmico
limiar_epidemico = media + (k * desvio_padrao)

# Resultados
print(f"Média da Incidência: {media:.2f}")
print(f"Desvio Padrão da Incidência: {desvio_padrao:.2f}")
print(f"Limiar Epidêmico (k={k}): {limiar_epidemico:.2f} casos por 100.000 habitantes")


Média da Incidência: 33.90
Desvio Padrão da Incidência: 6.20
Limiar Epidêmico (k=1): 40.10 casos por 100.000 habitantes
